In [1]:
import cv2 
import os
import numpy as np

In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

In [3]:
train_path = 'dataset/train'
train_images_type = os.listdir(train_path)

train_images = []
train_images_idx = []

for index, face_type in enumerate(train_images_type):
    img_path = f"{train_path}/{face_type}"
    
    for imgs in os.listdir(img_path):
        full_path = f"{img_path}/{imgs}"
        img = cv2.imread(full_path)
        width = int(img.shape[1] * 0.2)
        height = int(img.shape[0] * 0.2)
        img = cv2.resize(img, (width, height))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        face_rect = face_cascade.detectMultiScale(gray, 1.3, 3)
        if (len(face_rect) < 1): continue 

        for (x, y, w, h) in face_rect:
            face = gray[y:y+h, w:w+h]
            train_images.append(face)
            train_images_idx.append(index)

In [5]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(train_images, np.array(train_images_idx))

In [14]:
test_path = 'dataset/test'

for images in os.listdir(test_path):
    full_image_path = f"{test_path}/{images}"
    img = cv2.imread(full_image_path)
    # width = int(img.shape[1] * 0.08)
    # height = int(img.shape[0] * 0.08)
    img = cv2.resize(img, (width, height))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    detected_faces = face_cascade.detectMultiScale(gray, 1.2, 3)
    print(f"{len(detected_faces)} detected_faces")

    for (x, y, w, h) in detected_faces:
        if (len(detected_faces) < 1): 
            # no faces detected
            cv2.putText(img, "No Face Found", (50, 50), cv2.FONT_HERSHEY_PLAIN, 5, (0,0,255), 3)

        else:
            face_img = gray[y:y+h, x:x+w]
            smile_rect, reject_level, level_weight = smile_cascade.detectMultiScale3(face_img, 2.5, 20, outputRejectLevels=True)

            result, confidence = face_recognizer.predict(face_img)

            cv2.rectangle(img, (x, y), (x+w, y+h), (0,255,0), 3)
            text = f"{[result]} - {confidence}"
            cv2.putText(img, text, (x, y+h+15), cv2.FONT_HERSHEY_PLAIN, 0.5, (0,255,0))

            if len(level_weight) == 0:
                cv2.putText(img, "Not Smiling", (x+3, y+h-5), cv2.FONT_HERSHEY_PLAIN, 0.9, (0,0,255), 1)
            else:
                if max(level_weight) < 0:
                    cv2.putText(img, "Not Smiling", (x+3, y+h-15), cv2.FONT_HERSHEY_PLAIN, 0.9, (0,0,255), 1)
                else:
                    cv2.putText(img, "Smiling", (x+3, y+h-15), cv2.FONT_HERSHEY_PLAIN, 0.9, (0,255,0), 1)

    cv2.imshow(f"Image {images}", img)
    cv2.waitKey(0)

cv2.destroyAllWindows()
cv2.waitKey(0)

1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces
1 detected_faces


-1